In [1]:
import tensorflow as tf
import os
import pathlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from tensorflow import keras
from tensorflow.keras import models, layers, optimizers
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report


from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Check for TensorFlow GPU access
print(tf.config.list_physical_devices())

# See TensorFlow version
print(tf.__version__)

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
2.10.0


In [2]:
base_dir = '../data/fruits-360/'
test_dir = os.path.join(base_dir, 'Test')
train_dir = os.path.join(base_dir, 'Training')

train_dir = pathlib.Path(train_dir)
test_dir = pathlib.Path(test_dir) 

#Img size

img_height = 299
img_width = 299

In [26]:
# Data preprocessing - ImageDataGenerator

# Normalize the pixels in the train data images, resize and augment the data
train_datagen = ImageDataGenerator(
    rescale=1./255,# Image augmentaion 
    shear_range=0.2,
    zoom_range=0.2, # Zoom in on image by 20%
    horizontal_flip=True, #  Flip the image horizontally
    validation_split=0.2
    ) # Split 20% of the data for validation

# Normalize the test data images
test_datagen = ImageDataGenerator(rescale=1./255) 

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=32,
    shuffle=True,
    class_mode='categorical',
    subset='training') # Set as training data

validation_generator = train_datagen.flow_from_directory(
    train_dir, # Same directory as training data
    target_size=(img_height, img_width),
    batch_size=32,
    class_mode='categorical',
    subset='validation') # Set as validation data

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=32,
    class_mode='categorical')

Found 54190 images belonging to 131 classes.
Found 13502 images belonging to 131 classes.
Found 22688 images belonging to 131 classes.


In [16]:
# Only 13 clases - 10% of the data

# Data preprocessing - ImageDataGenerator

# Normalize the pixels in the train data images, resize and augment the data
train_datagen = ImageDataGenerator(
    rescale=1./255,# Image augmentaion 
    shear_range=0.2,
    zoom_range=0.2, # Zoom in on image by 20%
    horizontal_flip=True, #  Flip the image horizontally
    validation_split=0.2) # Split 20% of the data for validation

# Normalize the test data images
test_datagen = ImageDataGenerator(rescale=1./255) 

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=32,
    shuffle=True,
    classes=['Apple Red 2', 'Avocado', 'Banana', 'Cherry 2', 'Kiwi', 'Lemon', 'Mandarine', 'Peach', 'Pineapple', 'Raspberry', 'Strawberry', 'Tomato 1', 'Watermelon'],
    class_mode='categorical',
    subset='training') # Set as training data

validation_generator = train_datagen.flow_from_directory(
    train_dir, # Same directory as training data
    target_size=(img_height, img_width),
    batch_size=32,
    classes=['Apple Red 2', 'Avocado', 'Banana', 'Cherry 2', 'Kiwi', 'Lemon', 'Mandarine', 'Peach', 'Pineapple', 'Raspberry', 'Strawberry', 'Tomato 1', 'Watermelon'],
    class_mode='categorical',
    subset='validation') # Set as validation data

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=32,
    classes=['Apple Red 2', 'Avocado', 'Banana', 'Cherry 2', 'Kiwi', 'Lemon', 'Mandarine', 'Peach', 'Pineapple', 'Raspberry', 'Strawberry', 'Tomato 1', 'Watermelon'],
    class_mode='categorical')

Found 5421 images belonging to 13 classes.
Found 1351 images belonging to 13 classes.
Found 2268 images belonging to 13 classes.


In [ ]:
# Create the model based on the InceptionV3 architecture

# InceptionV3 model and use the weights from imagenet
conv_base = keras.applications.InceptionV3(
            include_top=False,
            weights="imagenet",
            input_shape=(img_height, img_width, 3)
)

conv_base.trainable = False # Freeze the base model layers

InceptionV3_model = conv_base.output
pool = GlobalAveragePooling2D()(InceptionV3_model)
dense_1 = layers.Dense(512, activation = 'relu')(pool)
output = layers.Dense(131, activation = 'softmax')(dense_1)

# Create an example of the Archictecture to plot on a graph
model_inception = models.Model(inputs=conv_base.input, outputs=output)

model_inception.summary()

In [ ]:
# Compile the model
model_inception.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
with tf.device("/device:GPU:0"):
    history = model_inception.fit(
        train_generator,
        epochs=10,
        validation_data=validation_generator,
        verbose = 1,
        callbacks=[EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)])
    
# Save the model
def save_model(model, seed):
    model.save(f"../models/inceptionv3-{seed}")
    
save_model(model_inception, 4)

In [32]:
# Test the model

loss, accuracy = model_inception.evaluate(test_generator)
print(f'Test accuracy: {accuracy:.3f}, Test loss: {loss:.3f}')

2022-12-13 15:54:59.116571: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


255/709 [=========>....................] - ETA: 1:15 - loss: 0.1786 - accuracy: 0.9531

KeyboardInterrupt: 

In [27]:
model_inception = keras.models.load_model('../models/inceptionv3-4')
class_dict = test_generator.class_indices # a dictionary of the form class name: class index

#model_inception = keras.models.load_model('../models/inceptionv3-3-13classes')
#class_dict = test_generator.class_indices # a dictionary of the form class name: class index


In [36]:
import tensorflow as tf
from tensorflow import keras
from PIL import Image
import json

from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing import image

# with open('class_names.json', 'w') as f:
#    json.dump(class_dict, f)

def classify(img_path):    
    img = image.load_img(img_path, target_size=(299, 299))
        
    img_array = image.img_to_array(img)

    img_batch = np.expand_dims(img_array, axis=0)

    img_preprocessed = preprocess_input(img_batch)
    
    prediction = model_inception.predict(img_preprocessed)

    return prediction

pred = classify('../random-image-test/kiwi_white.png')

score = tf.nn.softmax(pred[0])

klass = [k for k, v in class_dict.items() if v == np.argmax(score)][0]

print(f'Predicted class: {klass}')

1/1 [==============================] - 0s 33ms/step
Predicted class: Huckleberry


In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(1, len(history.history['accuracy']) + 1)

plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend()
plt.title('Training and Validation Accuracy')

plt.savefig('../images/inceptionv3-4-1-Accuracy.png')

plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend()
plt.title('Training and Validation Loss')

plt.savefig('../images/inceptionv3-4-2-Loss.png')